In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from koopmaneigen.matrix_eigsolver import MatrixEigSolver

dictionary basis = $\{x, y , x^2, y^2, xy\}$

In [3]:
def normalize(v):
    return v/np.linalg.norm(v)

taking linear system $x_{n+1} = Ax_n$ where A is 
$\begin{bmatrix}
    0.9 & 0.3 \\
    0 & 0.6
\end{bmatrix}$

koopman eigenfunctions will be $\phi(x,y) = (x-y)$ with eigenvalue 0.9 and $\phi(x,y) = y$ with eigenvalue 0.6

### set up koopman matrix

In [220]:
lambda_1 = 0.9
lambda_2 = 0.6

In [221]:
### representing eigenfunctions in dictionary sopace

In [666]:
v_1 = [1,-1,0,0,0]; d_1 = lambda_1
v_2 = [0,0,1,1,-2]; d_2 = (lambda_1)**2
v_3 = [0,1,0,0,0]; d_3 = lambda_2
v_4 = [0,0,0,-1,1]; d_4 = lambda_1 * lambda_2
v_5 = [0,0,0,1,0]; d_5 = (lambda_2)**2

In [667]:
V = np.array([normalize(v) for v in [v_1,v_2, v_3, v_4, v_5]]).T
print(V)

[[ 0.70710678  0.          0.          0.          0.        ]
 [-0.70710678  0.          1.          0.          0.        ]
 [ 0.          0.40824829  0.          0.          0.        ]
 [ 0.          0.40824829  0.         -0.70710678  1.        ]
 [ 0.         -0.81649658  0.          0.70710678  0.        ]]


In [224]:
np.linalg.cond(V)

6.288429268634205

In [225]:
D = np.diag([d_1, d_2, d_3, d_4, d_5])
print(D)

[[0.9  0.   0.   0.   0.  ]
 [0.   0.81 0.   0.   0.  ]
 [0.   0.   0.6  0.   0.  ]
 [0.   0.   0.   0.54 0.  ]
 [0.   0.   0.   0.   0.36]]


In [226]:
np.linalg.inv(V)

array([[ 1.41421356,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.        , -0.        ,  2.44948974, -0.        , -0.        ],
       [ 1.        ,  1.        ,  0.        ,  0.        ,  0.        ],
       [-0.        , -0.        ,  2.82842712, -0.        ,  1.41421356],
       [ 0.        ,  0.        ,  1.        ,  1.        ,  1.        ]])

In [227]:
# use pseudo inversion with regularization for inversion as it is more stable

In [228]:
np.linalg.pinv(V, rcond=1e-8)

array([[ 1.41421356e+00,  2.23711432e-17,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.44948974e+00,
        -1.97479288e-16,  4.49130194e-16],
       [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.82842712e+00,
         1.71222890e-16,  1.41421356e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00]])

In [229]:
# get transpose of koopman matrix
K_transpose = V @ D @ np.linalg.pinv(V, rcond=1e-8)
print(K_transpose)

[[ 9.00000000e-01  1.42369083e-17  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-3.00000000e-01  6.00000000e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  8.10000000e-01 -6.53026712e-17
   1.48518873e-16]
 [ 0.00000000e+00  0.00000000e+00  9.00000000e-02  3.60000000e-01
  -1.80000000e-01]
 [ 0.00000000e+00  0.00000000e+00 -5.40000000e-01  1.95984690e-16
   5.40000000e-01]]


In [230]:
### looks like a tridiagonal matrix?

In [231]:
eigsolver = MatrixEigSolver(K_transpose)

### Power method on K_transpose

In [232]:
eigvec, eigvalue = eigsolver.power_iteration(K_transpose)
print(eigvec)
print(eigvalue)


[ 7.07106781e-01 -7.07106781e-01  8.10946011e-06  8.10946011e-06
 -1.62189202e-05]
0.8999999999644875


first eigenvector is calculated correctly by power iteration

### try assymetric power deflation algorithm

In [233]:
eigs = eigsolver.power_iteration_with_deflation_asymm(num_eigen=5, num_iterations = 100000, tolerance=1e-10) 
with np.printoptions(precision=4, suppress=True):
    for i, (eigvec,eigvalue) in enumerate(eigs):
        print(f"eigenvalue: {eigvalue: .4}")
        print(f"eigenvalue error: {eigvalue - D[i,i]:.4}")
        eigvec = eigvec.reshape(eigvec.shape[0])
        print(f"eigenvector norm error: {np.linalg.norm(eigvec - V[:,i]):.4}")
        print("--------")

eigenvalue:  0.9
eigenvalue error: -4.023e-10
eigenvector norm error: 6.686e-05
--------
eigenvalue:  0.81
eigenvalue error: 4.422e-10
eigenvector norm error: 4.176e-05
--------
eigenvalue:  0.6
eigenvalue error: -3.784e-10
eigenvector norm error: 7.771e-05
--------
eigenvalue:  0.54
eigenvalue error: 3.443e-10
eigenvector norm error: 4.055e-05
--------
eigenvalue:  0.36
eigenvalue error: -5.551e-17
eigenvector norm error: 2.523e-09
--------


### complex eigenvalues

In [545]:
lambda_1 = 0.9 + 0.2j
lambda_2 = 0.9 - 0.2j
lambda_3 = 0.6 + 0.1j
lambda_4 = 0.6 - 0.1j
lambda_5 = 0.3

In [546]:
v_1 = [1+2j, 1 + 1j, 0, 0, 1 + 1j]; v_1 = normalize(v_1)
v_2 = np.conjugate(v_1);
v_3 = [0, 1 + 1j, 1, 3-2j, 0]; v_3 = normalize(v_3)
v_4 = np.conjugate(v_3)
v_5 = [1, 0 ,0, 0, 1]

In [547]:
V = np.array([normalize(v) for v in [v_1,v_2, v_3, v_4, v_5]]).T
print(V)

[[0.33333333+0.66666667j 0.33333333-0.66666667j 0.        +0.j
  0.        -0.j         0.70710678+0.j        ]
 [0.33333333+0.33333333j 0.33333333-0.33333333j 0.25      +0.25j
  0.25      -0.25j       0.        +0.j        ]
 [0.        +0.j         0.        -0.j         0.25      +0.j
  0.25      -0.j         0.        +0.j        ]
 [0.        +0.j         0.        -0.j         0.75      -0.5j
  0.75      +0.5j        0.        +0.j        ]
 [0.33333333+0.33333333j 0.33333333-0.33333333j 0.        +0.j
  0.        -0.j         0.70710678+0.j        ]]


#### create a complex matrix using blocks

In [549]:
C = np.vstack((np.array(v_1).real, np.array(v_1).imag, np.array(v_3).real, np.array(v_3).imag, np.array(v_5))).T
C, np.linalg.matrix_rank(C)

(array([[ 0.33333333,  0.66666667,  0.        ,  0.        ,  1.        ],
        [ 0.33333333,  0.33333333,  0.25      ,  0.25      ,  0.        ],
        [ 0.        ,  0.        ,  0.25      ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.75      , -0.5       ,  0.        ],
        [ 0.33333333,  0.33333333,  0.        ,  0.        ,  1.        ]]),
 5)

In [550]:
B_1 = np.array([[lambda_1.real, lambda_1.imag], [-lambda_1.imag, lambda_1.real]])
B_2 = np.array([[lambda_3.real, lambda_3.imag, 0], [-lambda_3.imag, lambda_3.real, 0]])
B_3 = np.array([lambda_5])

B = np.block([[B_1, np.zeros_like(B_2)],
                   [np.zeros_like(B_1), B_2]])
B = np.vstack((B, np.array([0,0,0,0,lambda_5])))
print(B)

[[ 0.9  0.2  0.   0.   0. ]
 [-0.2  0.9  0.   0.   0. ]
 [ 0.   0.   0.6  0.1  0. ]
 [ 0.   0.  -0.1  0.6  0. ]
 [ 0.   0.   0.   0.   0.3]]


Koopman matrix should only have real entries

In [551]:
K_transpose_comp = C @ B @ np.linalg.pinv(C, rcond=1e-8)
with np.printoptions(precision=4, suppress=True):
    print(K_transpose_comp)

[[ 1.5   0.2  -0.5   0.1  -1.2 ]
 [ 0.4   0.7  -0.2  -0.   -0.4 ]
 [ 0.    0.    0.75 -0.05 -0.  ]
 [ 0.   -0.    0.65  0.45 -0.  ]
 [ 0.4   0.4  -1.    0.2  -0.1 ]]


In [552]:
np.linalg.matrix_rank(K_transpose_comp), np.linalg.cond(K_transpose_comp)

(5, 20.80802704676878)

In [553]:
eigsolver_c = MatrixEigSolver(K_transpose_comp)

#### run modified power iteration

In [554]:
eigvec, eigval = eigsolver_c.power_iteration_complex_eigs(K_transpose_comp)
eigval, eigvec

max iter reached in power method 0.0
tol achieved in 1 iterations


((0.9000000000043709+0.19999999996474102j),
 array([-7.39942504e-01-8.96696479e-02j, -4.61899432e-01+9.41867119e-02j,
        -6.58502766e-11-1.00852021e-10j, -1.29423959e-10-7.31546288e-11j,
        -4.61899432e-01+9.41867121e-02j]))

In [555]:
# check if eigen equation satisfied
np.linalg.norm(K_transpose_comp@eigvec - eigval*eigvec)

8.226326325796593e-11

In [556]:
eigvec, eigval = eigsolver_c.power_iteration_complex_eigs(K_transpose_comp.T)
eigval, eigvec

max iter reached in power method 0.0
tol achieved in 1 iterations


((0.8999999999265668+0.20000000000246718j),
 array([-0.37142441-0.18962319j,  0.2805238 -0.09090061j,
        -0.70130949+0.22725153j,  0.1402619 -0.04545031j,
         0.37142441+0.18962319j]))

#### run the same method on the matrix with real eigenvalues used earlier

In [1102]:
eigvec, eigval = eigsolver.power_iteration_complex_eigs(K_transpose)
eigval, eigvec

max iter reached in power method 0.0
tol achieved in 2 iterations


((0.9000000000000623+0j),
 array([ 7.07106781e-01+0.j, -7.07106781e-01+0.j, -4.34246957e-11+0.j,
        -2.32096590e-11+0.j,  6.66343491e-11+0.j]))

In [1103]:
# check if eigen equation satisfied
np.linalg.norm(K_transpose@eigvec - eigval*eigvec)

5.188467955956665e-12

#### run the algorithm with deflation

In [1032]:
eigs, left_eigs = eigsolver_c.power_iteration_with_deflation_asymm_complex(num_eigen=K_transpose_comp.shape[0], 
                                                 num_iterations=10000, tolerance=1e-10)

eigenvalue 1
max iter reached in power method 0.0
tol achieved in 1 iterations
max iter reached in power method 0.0
tol achieved in 93 iterations
lambda:  (0.8999999999466716+0.20000000000144114j)
---------------------------------
eigenvalue 2
max iter reached in power method 0j
tol achieved in 1 iterations
max iter reached in power method 0j
tol achieved in 1 iterations
lambda:  (0.6+0.09999999999999964j)
---------------------------------
eigenvalue 3
tol achieved in 1 iterations
tol achieved in 1 iterations
lambda:  (0.3+0j)
---------------------------------


In [657]:
def check_complex_vec_equal(x,y):
    # equality in cauchy schwarz
    return np.allclose(np.abs(x.conj().T @ y), np.linalg.norm(x) * np.linalg.norm(y))

In [658]:
with np.printoptions(precision=4, suppress=True):
    for i, (eigvec,eigvalue) in enumerate(eigs):
        print(f"eigenvalue: {eigvalue: .4}")
        print(f"eigenvalue error: {np.abs(eigvalue - eval(f'lambda_{i+1}')):.4}")
        eigvec = eigvec.reshape(eigvec.shape[0])
        print(f"eigenvector equal to explicit eigvector: {check_complex_vec_equal(V[:,i], eigvec)}")
        print("--------")

eigenvalue: ( 0.9+0.2j)
eigenvalue error: 8.345e-11
eigenvector equal to explicit eigvector: True
--------
eigenvalue: ( 0.9-0.2j)
eigenvalue error: 8.345e-11
eigenvector equal to explicit eigvector: True
--------
eigenvalue: ( 0.6+0.1j)
eigenvalue error: 6.621e-16
eigenvector equal to explicit eigvector: True
--------
eigenvalue: ( 0.6-0.1j)
eigenvalue error: 6.621e-16
eigenvector equal to explicit eigvector: True
--------
eigenvalue: ( 0.3+0j)
eigenvalue error: 5.551e-17
eigenvector equal to explicit eigvector: True
--------


#### run with matrix with real eigenvalues

In [1091]:
eigs, left_eigs = eigsolver.power_iteration_with_deflation_asymm_complex(num_eigen=K_transpose.shape[0], 
                                                 num_iterations=10000, tolerance=1e-10)

eigenvalue 1
max iter reached in power method 0.0
tol achieved in 2 iterations
max iter reached in power method 0.0
tol achieved in 2 iterations
lambda:  (0.9000000000000425+0j)
---------------------------------
eigenvalue 2
max iter reached in power method 0j
tol achieved in 2 iterations
max iter reached in power method 0j
tol achieved in 2 iterations
lambda:  (0.8099999999999992+0j)
---------------------------------
eigenvalue 3
max iter reached in power method 0j
tol achieved in 1 iterations
max iter reached in power method 0j
tol achieved in 1 iterations
lambda:  (0.5999999999999998+0j)
---------------------------------
eigenvalue 4
max iter reached in power method 0j
tol achieved in 1 iterations
max iter reached in power method 0j
tol achieved in 1 iterations
lambda:  (0.5399999999999998+0j)
---------------------------------
eigenvalue 5
tol achieved in 1 iterations
tol achieved in 1 iterations
lambda:  (0.35999999999999993+0j)
---------------------------------


In [1101]:
with np.printoptions(precision=4, suppress=True):
    for i, (eigvec,eigvalue) in enumerate(eigs):
        print(f"eigenvalue: {eigvalue: .4}")
        assert np.all(eigvec.imag == 0)
        assert np.all(eigvalue.imag ==0)
        print("eigenvalue and eigenvector are real")
        
        print(f"eigenvalue error: {eigvalue - D[i,i]:.4}")
        eigvec = eigvec.reshape(eigvec.shape[0])
        assert np.all(eigvec.imag == 0)
        print(f"eigenvector norm error: {np.linalg.norm(eigvec - V[:,i]):.4}")
        print("--------")

eigenvalue: ( 0.9+0j)
eigenvalue and eigenvector are real
eigenvalue error: (4.252e-14+0j)
eigenvector norm error: 5.95e-11
--------
eigenvalue: ( 0.81+0j)
eigenvalue and eigenvector are real
eigenvalue error: (-8.882e-16+0j)
eigenvector norm error: 1.614e-11
--------
eigenvalue: ( 0.6+0j)
eigenvalue and eigenvector are real
eigenvalue error: (-2.22e-16+0j)
eigenvector norm error: 1.934e-10
--------
eigenvalue: ( 0.54+0j)
eigenvalue and eigenvector are real
eigenvalue error: (-2.22e-16+0j)
eigenvector norm error: 1.262e-10
--------
eigenvalue: ( 0.36+0j)
eigenvalue and eigenvector are real
eigenvalue error: (-5.551e-17+0j)
eigenvector norm error: 3.67e-16
--------
